In [95]:
import tkinter
import random
from math import sqrt

In [105]:
# generate one wire according to section 2.1 of paper
# returns list of [p1, p2] where p1 and p2 are tuples
def generate_wire(l):
    
    # q1 and q2 are distinct values of random var Q
    # Q is discrete cont. random var over [0,3]
    # the purpose of q1 and q2 is to "anchor" a wire's endpoints 
    # to distinct sides of the simulation
    q1 = -1
    q2 = -1
    
    while q1 == q2:
        q1 = random.randint(0,3)
        q2 = random.randint(0,3)
        
#     print("Q values: ", q1, q2)
    
    # x1, y1, x2, y2 are the endpoints of the wire
    # they are from uniform rand var T over [0, l]
    x1 = random.uniform(0, l)
    x2 = random.uniform(0, l)
    y1 = random.uniform(0, l)
    y2 = random.uniform(0, l)
    
#     print("Initial endpoints:", x1, y1, x2, y2)
    
    # apply the adjustment table to q1, q2 to anchor endpoints
    # @TODO clean this code up
    if q1 == 0:
        y1 = 0
    elif q1 == 1:
        x1 = 0
    elif q1 == 2:
        y1 = l
    elif q1 == 3:
        x1 = l
        
    # handle q2
    if q2 == 0:
        y2 = 0
    elif q2 == 1:
        x2 = 0
    elif q2 == 2:
        y2 = l
    elif q2 == 3:
        x2 = l
        
    p1 = (x1, y1)
    p2 = (x2, y2)
    
#     print("P1:", p1)
#     print("P2:", p2)
    
    return[p1, p2]
    
            

    

In [112]:
# determine if an electrode is touching a wire
# takes in tuple of coordinates of electrode
# takes in list of tuples of endpoints of wire
def electrode_touching_wire(electrode_points, wire_points, r_e):
    # the formula for min distance between wire and electrode is given in sec 2.1
    
    x_e, y_e = electrode_points
    x1, y1 = wire_points[0]
    x2, y2 = wire_points[1]
    
    numerator = abs(x_e * (y2 - y1) - y_e * (x2 - x1) + x2 * y1 - y2 * x1)
    denominator = sqrt((x2 - x1)**2 + (y2 - y1)**2)
    
    distance = numerator / denominator
    
    # wire is touching if min distance is less than electrode radius
    return distance <= r_e

In [127]:
# main function to run simulation
def main(a=1, r_e=0.4, n_e=9, lambda_constant=30):
    
    # delete???
    r_e = r_e
    
    # height/width of grid
    l = a * (1 + sqrt(n_e)) 
    
    # number of wires
    n_w = int(lambda_constant * sqrt(n_e))
    
    # generate wire coordinates
    wires = []
    for i in range(n_w):
        wires.append(generate_wire(l))
        
    # generate electrode coordinates
    # they are evenly spaced out along the lxl grid
    electrodes = []
    for i in range(int(sqrt(n_e))):
        for j in range(int(sqrt(n_e))):
            x = i * (l / n_e)
            y = j * (l / n_e)
            electrodes.append((x, y))
        
    # generate dictionaries of electrodes and the wires they touch
    electrode_wire_dict = {}
    
    for electrode in electrodes:
        for wire in wires:
            wires_touching = []
            if electrode_touching_wire(electrode, wire, r_e):
                wires_touching.append(wire)
            electrode_wire_dict[electrode] = wires_touching
    
    print(electrode_wire_dict)

In [128]:
# cell for testing
main()

NameError: name 'r_e' is not defined